### Problem Statement

Use Case: Developing an LLM chatbot that would give product details

Requirements: 
*  Name of the product. 
*  Description of the product. 
*  Price of the product in USD

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()